# Part 6 - Recommender Systems

#### Name: Amirhossein Kiani
#### Contact: ahosseinkiani@gmail.com

In the last notebook we preprocessed most of the text data associated with our recipes and used `OpenAI`'s embedding model `ada-002` to extract semantics embeddings from them. We used the obtained embeddings to create a basic recommender system, which seems fairly successful in its performance.

In this notebook, we will import the recipes dataset enhanced with BerTopic-extracted topics and the semantic embeddings obtained from previous notebooks to further extend the basic recommender system from Notebook 6 with more features. We will also propose another recommender system that takes into account nutritional facts associated with recipes. We will also curate a dataset for a Streamlit app that brings our semantic-based recommender system to life.

### Table of Contents:

- [Imports and Sanity Checks](#imports)
- [The Semantic-Based Recommender System](#semantic)
- [A Nutrition-Based Recommender System ](#nutr)
- [Ensembling the Recommender Systems](#ensemble)
- [Curating the Data for Streamlit](#curate)
- [Conclusion](#conc)

## Import and Sanity Checks <a class ='author' id='imports'></a>

In [1]:
# Checking if I'm in the right environment:
import sys
sys.executable

'C:\\Users\\mathe\\anaconda3\\envs\\deepchef\\python.exe'

In [1]:
# General libraries:

import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
# This allows scrolling through all the columns. Useful for dataframes with too many columns

pd.set_option('display.max_columns', None)

In [3]:
recipes = pd.read_pickle('recipes_with_ada_embeddings.pkl')

In [5]:
recipes.sample()

,RecipeId,Name,AuthorId,AuthorName,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeInstructions,url,ingred_quants,ingred_items,YearPublished,MonthPublished,DayPublished,HourPublished,TotalMinutes,CorrectAggregatedRating,BerTopics,CleanRecipeTextData,num_tokens,ada_embeddings
92535,467077.0,Austin's Hyde Park Bar &amp; Grill Mac and Cheese,160756,Colean,Look for the giant fork outside of this popula...,[https://img.sndimg.com/food/image/upload/w_55...,Quick and Easy,[],"[1 1⁄2, 5, 1⁄2, 1⁄2, 1⁄4, 1⁄2, None, 1, None]","[heavy cream, cheese, cheese, cheese, kosher s...",1.0,462.3,26.5,16.1,81.2,514.5,38.0,1.2,6.2,18.1,12.0,"[Heat cream and milk in a double boiler., Add ...",https://www.food.com/recipe/Austin's-Hyde-Park...,"[1 1⁄2, 5, 1⁄2, 1⁄2, 1⁄4, 1⁄2, , 1, ]","[cups heavy cream, cups whole milk, lb cheese,...",2011,10,30,21,25,4.0,10_macaroni_elbow_sharp,quick easicups heavy cream cups whole milk lb ...,89,"[0.008834578096866608, 0.0013761422596871853, ..."


In [6]:
recipes = recipes.reset_index(drop=True)

In [11]:
recipes.sample()

,RecipeId,Name,AuthorId,AuthorName,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeInstructions,url,ingred_quants,ingred_items,YearPublished,MonthPublished,DayPublished,HourPublished,TotalMinutes,CorrectAggregatedRating,BerTopics,CleanRecipeTextData,num_tokens,ada_embeddings
71438,351167.0,West Coast Cauliflower,498271,loof751,My daughter recently bought me some vintage co...,[https://img.sndimg.com/food/image/upload/w_55...,Vegetables,"[Vegetable, < 30 Mins]","[1, 1, 1, 1⁄4, 1⁄4, 1⁄8, 2, 2]","[frozen cauliflower, milk, salt, pepper, shrimp]",2.0,270.2,6.8,1.8,246.4,1589.5,18.9,5.4,5.3,33.2,4.0,"[Cook cauliflower and peas in 1"" boiling water...",https://www.food.com/recipe/West-Coast-Caulifl...,"[1, 1, 1, 1⁄4, 1⁄4, 1⁄8, 2, 2]","[(10 ounce) package frozen cauliflower, (10 ou...",2009,1,21,15,20,5.0,-1_pepper_onion_garlic,vegetVegetable < 30 Mins(10 ounce) package fro...,62,"[0.011134611442685127, -0.025623183697462082, ..."


## The Semantic-Based Recommender System <a class ='author' id='semantic'></a>

In this section we will set our API key and send a request to verify that we can send a request.
To do so, you need to get yourself a private API key first by signing up to create your account.

**NOTE:** we have discussed this recommender system in the [previous notebook](https://github.com/amirkiaml/DeepChef-BSTN-Capstone/blob/main/Part%206%20-%20Semantic%20Embeddings.ipynb). Not much new material is said here, so feel free to move on the the nexr section if you have gone through the previous notebook.

In [11]:
import tiktoken #pip install
import openai   #pip install

# Copy and paste your personal api key inbetween the quotation marks and then execute the below code
openai.api_key = "..."

In [12]:
# list models as a quick test
models = openai.Model.list()

# print the first model's id
print(models.data[0].id)

babbage


In [13]:
# Test if we can send a request

completion = openai.Completion.create(model="ada", prompt="Hello world")
print(completion['choices'][0].text)

 through Ruby. This is what I would do in a coder that I basically


In [14]:
# Import `get_embedding`

from openai.embeddings_utils import get_embedding

In [15]:
# Embedding model parameters

embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # This is the encoding for text-embedding-ada-002
max_tokens = 8000

In [16]:
from tenacity import retry, stop_after_attempt, wait_exponential

# Define the retry strategy using Tenacity
@retry(stop=stop_after_attempt(5), wait=wait_exponential(multiplier=1, min=4, max=30))

def get_embedding(text, engine="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    response = openai.Embedding.create(input=[text], model=engine)
    embeddings = response['data'][0]['embedding']
    return embeddings

In [18]:
from heapq import nlargest

recipes['SimilarityRate'] = ''

def most_similar_recipes(user_input, df):
    """
    This function takes an input and sorts out the dataset based on cosine similarities to the user input.
    """
    # Step 1: Get the embeddings for the user_input
    input_embeddings = get_embedding(user_input)

    # Step 2: Calculate the similarity between user_input and each row in the DataFrame
    similarities = []

    for index, row in df.iterrows():
        cur_embeddings = row['ada_embeddings']
        similarity = cosine_similarity(np.array(input_embeddings).reshape(1, -1), np.array(cur_embeddings).reshape(1, -1))[0][0]
        similarities.append(similarity)

    recipes['SimilarityRate'] = similarities
    
    # Step 3: Return the DataFrame rows with sorted using SimilarityRate, from higher to lower
    return df.sort_values(['SimilarityRate'],ascending=False)[:5]

Let's now create a function that returns common words in two given strings. We can then apply it to our strings to find common words in them.

In [19]:
def find_common_words(str1, str2):
    # Tokenize the strings into words
    words1 = str1.split()
    words2 = str2.split()

    # Create sets of unique words
    set1 = set(words1)
    set2 = set(words2)

    # Find the common words using the intersection of the sets
    common_words = set1.intersection(set2)

    return common_words

**NOTE:** It's important to note that this function only gives us vizibility into the exact word matching between given strings, BUT the there's more to this when comparing them using the ada-002 embeddings: they capture the semantic of the terms in the strings as well. So naturally the string 'beef bacon grill' is more similar to 'chicken barbeque' than to 'potato char', even thouh none of these strings has any exact words in common with the other two. And all of these are closer to one another in comparsion to 'North Korea nuclear arsenal'. Let's take a look:

In [20]:
str1 = 'beef bacon grill'
str2 = 'chicken barbeque'
str3 = 'potato char'
str4 = 'North Korea nuclear arsenal'


strings = [str1,str2,str3,str4]

for i in strings:
    for j in strings:
        if i!=j:
            print(f"The similarity of '{i}' to '{j}' is: {cosine_similarity(np.array(get_embedding(i)).reshape(1, -1),np.array(get_embedding(j)).reshape(1, -1))[0,0]}")

The similarity of 'beef bacon grill' to 'chicken barbeque' is: 0.8538504508409608
The similarity of 'beef bacon grill' to 'potato char' is: 0.7905343323285426
The similarity of 'beef bacon grill' to 'North Korea nuclear arsenal' is: 0.729916782632251
The similarity of 'chicken barbeque' to 'beef bacon grill' is: 0.8538504508409608
The similarity of 'chicken barbeque' to 'potato char' is: 0.8165535221019657
The similarity of 'chicken barbeque' to 'North Korea nuclear arsenal' is: 0.7710235429971272
The similarity of 'potato char' to 'beef bacon grill' is: 0.7905014676783799
The similarity of 'potato char' to 'chicken barbeque' is: 0.8165319966416351
The similarity of 'potato char' to 'North Korea nuclear arsenal' is: 0.7253796081418088
The similarity of 'North Korea nuclear arsenal' to 'beef bacon grill' is: 0.7298206895796701
The similarity of 'North Korea nuclear arsenal' to 'chicken barbeque' is: 0.7711389191879254
The similarity of 'North Korea nuclear arsenal' to 'potato char' is: 

Now, let's test our functions to see if they work well. We can use the following sample user input:

In [3]:
input('What do you like?')

What do you like?a


'a'

In [21]:
import time

start_time = time.time()

user_input = (
    "Give me a recipe with chicken, bacon and eggs in it." 
)
top_5_similar_recipes = most_similar_recipes(user_input, recipes)

end_time = time.time()

top_5_similar_recipes

,RecipeId,Name,AuthorId,AuthorName,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeInstructions,url,ingred_quants,ingred_items,YearPublished,MonthPublished,DayPublished,HourPublished,TotalMinutes,CorrectAggregatedRating,BerTopics,CleanRecipeTextData,num_tokens,ada_embeddings,Caloriesfor1,FatContentfor1,SaturatedFatContentfor1,CholesterolContentfor1,SodiumContentfor1,CarbohydrateContentfor1,FiberContentfor1,SugarContentfor1,ProteinContentfor1,RecipeServingsfor1,SimilarityRate
71524,351613.0,BLT Egg Wrap,883095,Chef mariajane,"This is a quick and easy, and also delicious, ...",[https://img.sndimg.com/food/image/upload/w_55...,Breakfast/Brunch,"[Vegetable, < 15 Mins]","[1, 1, 1⁄4, None, 1, 2, 1, 2]","[egg, onion powder, low-fat tortilla, low-fat ...",1.0,78.2,4.8,1.6,186.0,73.9,1.8,0.4,0.8,6.6,1.0,[Cook bacon in non-stick skillet over medium h...,https://www.food.com/recipe/BLT-Egg-Wrap-351613,"[1, 1, 1⁄4, , 1, 2, 1, 2]","[slice chicken bacon or 1 slice turkey bacon, ...",2009,1,23,17,7,5.00,6_mayonnaise_boiled_crabmeat,breakfastbrunchVegetable < 15 Minsslice chicke...,97,"[-0.008318441919982433, -0.01105539035052061, ...",78.20,4.80,1.60,186.00,73.90,1.80,0.40,0.80,6.60,1.0,0.851192
79421,391828.0,Cobb Salad,1072593,gailanng,Make and share this Cobb Salad recipe from Foo...,[https://img.sndimg.com/food/image/upload/w_55...,Main Dish,"[Chicken, Poultry, Vegetable, Meat, Weeknight,...","[8, 3, 1, 3, 2, 3⁄4, 1, 1 1⁄2, 1, None]","[bacon, eggs, lettuce, chicken meat, tomatoes,...",7.0,173.3,12.4,4.7,105.7,296.1,8.6,4.2,2.5,9.1,6.0,"[Divide shredded lettuce among plates., Evenly...",https://www.food.com/recipe/Cobb-Salad-391828,"[8, 3, 1, 3, 2, 3⁄4, 1, 1 1⁄2, 1, ]","[slices bacon, cooked, drained and crumbled, e...",2009,9,25,14,10,4.14,-1_pepper_onion_garlic,main dishChicken Poultry Vegetable Meat Weekni...,74,"[-0.0035131655167788267, -0.00453499099239707,...",28.88,2.07,0.78,17.62,49.35,1.43,0.70,0.42,1.52,1.0,0.842568
26776,141154.0,Crock Pot Swiss Cheese Chicken,195916,TERRY B.,Make and share this Crock Pot Swiss Cheese Chi...,[https://img.sndimg.com/food/image/upload/w_55...,Chicken,"[Chicken, Poultry, Meat, Beginner Cook, Easy]","[2, 4, 4, 1, 1, 1, 1, 4]","[potatoes, boneless chicken breasts, garlic cl...",1.0,619.3,27.0,10.4,130.5,718.5,48.4,5.5,4.4,45.3,4.0,"[In skillet cook bacon until crispy, remove an...",https://www.food.com/recipe/Crock-Pot-Swiss-Ch...,"[2, 4, 4, 1, 1, 1, 1, 4]","[slices turkey bacon, potatoes, quartered, bon...",2005,10,12,16,380,5.00,-1_pepper_onion_garlic,chickenChicken Poultry Meat Beginner Cook Easy...,74,"[0.017721202224493027, -0.023005293682217598, ...",154.82,6.75,2.60,32.62,179.62,12.10,1.38,1.10,11.32,1.0,0.841542
10131,60641.0,Famous Eggs and Bacon!,80047,lilbubblegurl,Make and share this Famous Eggs and Bacon! rec...,[https://img.sndimg.com/food/image/upload/w_55...,Breakfast/Brunch,"[< 15 Mins, Easy]","[1⁄2, 1⁄8, 2, 1]","[milk, eggs, eggs, cheese]",7.0,130.7,9.4,5.2,112.2,307.4,2.9,0.0,0.2,8.6,4.0,[Put 1/4 cup of bacon in the bottom of the pan...,https://www.food.com/recipe/Famous-Eggs-and-Ba...,"[1⁄2, 1⁄8, 2, 1]","[cup cooked bacon, crumbled, divided, cup milk...",2003,4,22,20,7,3.86,56_blue_cheddar_cheese,"breakfastbrunch< 15 Mins Easycup cooked bacon,...",44,"[0.0049086022190749645, -0.009938988834619522,...",32.67,2.35,1.30,28.05,76.85,0.72,0.00,0.05,2.15,1.0,0.840941
79828,394201.0,Skillet Corn Scramble,542159,Debbwl,A hearty scrambled egg dish that is good for s...,[https://img.sndimg.com/food/image/upload/w_55...,Breakfast/Brunch,[< 15 Mins],"[5, 5, 1⁄3, 1⁄4, 1, 1⁄2, 1⁄4, 1⁄8, 2, 1 1⁄2, N...","[bacon, eggs, milk, green pepper, onion, salt,...",6.0,171.0,9.7,3.2,193.7,401.7,12.8,1.5,3.2,9.5,5.0,"[In fry pan, cook bacon until crisp., Drain on...",https://www.food.com/recipe/Skil

In [22]:
print(f"Runtime: {round(end_time-start_time,2)} seconds.")

Runtime: 64.38 seconds.


Let's inspect the first one:

In [23]:
top_5_similar_recipes.iloc[0]['CleanRecipeTextData']

'breakfastbrunchVegetable < 15 Minsslice chicken bacon or 1 slice turkey bacon, cooked egg teaspoon onion powder salt and pepper small low-fat tortilla teaspoons low-fat mayonnaise leaf lettuce tablespoons tomatoes, dicedcook bacon nonstick skillet medium heat microwav crisp fold half set aside wipe skillet clean whisk togeth egg onion powder salt pepper small bowl pour mixtur skillet cook stir slightli medium heat mxture begin set gentli move spatula across bottom side skillet form large soft curds cook egg thicken visibl liquid egg remain still moist spread tortilla mayonnaise place lettuc top add bacon egg tomato fold wrap serve'

It's needless to say how similar this is to our user input: 'chicken', 'bacon', and 'egg' appear in both.
   
Let's try a more complicated input: I'm just copying most of the text data addociated to a recipe from this external link: https://themodernproper.com/30-best-ground-beef-recipes

In [11]:
import gdown
print(gdown.__version__)

4.7.1


In [24]:
start_time = time.time()

user_input = (
"Easy Ground Beef Recipes with Few Ingredients Thai Basil Beef (cover photo). A speedy, savory Thai beef basil stir fry that’s\
just a bit spicy and really hits the spot. Gingery Ground Beef (Soboro Donburi). Five ingredients, a few minutes and a hot \
skillet, and you’ll be digging into a delicious soboro donburi, a gingery ground beef that reminds us that the best Japanese \
recipes are often the simplest. Crock-Pot Taco Meat. Just three-ingredients and the use of our trusty slow-cooker make this \
Crock-Pot beef taco meat recipe an easy win. Taco Pizza. Two classic family favorites, pizza and tacos, come together in this\
super easy weeknight meal that is destined to become a favorite in your home. Ground Beef Burger and Sandwich Recipes Classic\
Cheeseburger with Secret Sauce. A perfectly soft bun, quality, juicy ground beef that is seasoned well, plenty of cheese,\
caramelized onions and a really good secret sauce are the key to this truly classic cheeseburger recipe. Patty Melt. As diner\
sandwiches go, the melty, meaty, oniony perfection that is the patty melt reigns supreme—if you know, you know. And if you don’t\
know, well, welcome to the club. Meatball Sub Sandwich. Big juicy, tender meatballs simmered in marinara sauce (store-bought\
or homemade—either works!) stuffed into garlicky hoagie rolls, topped with mozzarella and broiled to melty, bubbly perfection.\
Beef Sliders. These juicy ground beef sliders—slathered with a homemade bright and lemony sun-dried tomato mayo, spicy pickles,\
and fresh arugula—prove that bigger doesn’t always equal better. Low-Carb Bacon Burger with Guacamole. You’ll forget all about\
the missing bun while devouring this juicy, keto-friendly bacon guacamole burger loaded with veggies and jalapeno aioli!"
)

top_5_similar_recipes = most_similar_recipes(user_input, recipes)

end_time = time.time()

top_5_similar_recipes

,RecipeId,Name,AuthorId,AuthorName,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeInstructions,url,ingred_quants,ingred_items,YearPublished,MonthPublished,DayPublished,HourPublished,TotalMinutes,CorrectAggregatedRating,BerTopics,CleanRecipeTextData,num_tokens,ada_embeddings,Caloriesfor1,FatContentfor1,SaturatedFatContentfor1,CholesterolContentfor1,SodiumContentfor1,CarbohydrateContentfor1,FiberContentfor1,SugarContentfor1,ProteinContentfor1,RecipeServingsfor1,SimilarityRate
43303,218875.0,Garlic Lover's Burger,178711,Buckeye Karen,"If you're like me, you just can't get enough o...",[https://img.sndimg.com/food/image/upload/w_55...,Meat,"[< 15 Mins, Beginner Cook, Easy]","[1, 1⁄8, 3, 1⁄4, 3, 1 1⁄2, None, 4, None, None...","[ground beef, dried cilantro, garlic, honey, s...",1.0,449.9,19.0,7.1,77.1,1039.0,42.4,1.3,20.5,27.3,4.0,[Mix all ingredients together (except ground p...,https://www.food.com/recipe/Garlic-Lover's-Bur...,"[1, 1⁄8, 3, 1⁄4, 3, 1 1⁄2, , 4, , , , ]","[lb ground beef (or chuck or sirloin), cup dri...",2007,3,26,20,15,5.0,-1_pepper_onion_garlic,meat< 15 Mins Beginner Cook Easylb ground beef...,170,"[-0.007803387939929962, 0.002627050504088402, ...",112.48,4.75,1.78,19.27,259.75,10.60,0.32,5.12,6.82,1.0,0.893549
107680,536579.0,Korean Ground Beef and Rice Bowls,972011,Yearbook Mom,Make and share this Korean Ground Beef and Ric...,[https://img.sndimg.com/food/image/upload/w_55...,Quick and Easy,[Easy],"[1, 3, 1⁄4, 1⁄4, 2, 1⁄4, 1⁄4, 1⁄4, 2, 1⁄4, 1]","[lean ground beef, garlic cloves, brown sugar,...",NaN,407.5,13.8,5.0,73.7,611.4,42.9,0.7,13.8,26.0,4.0,[1. In a large skillet cook the ground beef an...,https://www.food.com/recipe/Korean-Ground-Beef...,"[1, 3, 1⁄4, 1⁄4, 2, 1⁄4, 1⁄4, 1⁄4, 2, 1⁄4, 1]","[lb lean ground beef (90% lean), garlic cloves...",2018,7,30,18,20,NaN,-1_pepper_onion_garlic,quick easiEasylb lean ground beef (90% lean) g...,89,"[-0.003680585417896509, -0.00722745293751359, ...",101.88,3.45,1.25,18.42,152.85,10.72,0.18,3.45,6.50,1.0,0.890112
104057,525697.0,Thai Basil Beef,403989,Papagayita,"Delicious, adaptable recipe that's great for u...",[https://img.sndimg.com/food/image/upload/w_55...,Meat,"[Thai, Asian, Weeknight, < 30 Mins, Easy]","[1, 2, 3 -4, 1, 1, 1 -2, 1, 1, 2, 1⁄4, 1, 1⁄2,...","[ground beef, garlic cloves, onion, bell peppe...",1.0,302.6,19.7,7.1,77.6,929.4,7.6,1.3,3.7,23.1,4.0,[Heat a large frying pan over medium high heat...,https://www.food.com/recipe/Thai-Basil-Beef-52...,"[1, 2, 3 -4, 1, 1, 1 -2, 1, 1, 2, 1⁄4, 1, 1⁄2, ]","[lb ground beef, teaspoons sesame oil, garlic ...",2016,1,24,15,25,5.0,-1_pepper_onion_garlic,meatThai Asian Weeknight < 30 Mins Easylb grou...,139,"[-0.004620139487087727, -0.007158639840781689,...",75.65,4.92,1.78,19.40,232.35,1.90,0.32,0.92,5.78,1.0,0.888851
78319,385719.0,Spanish Rice With Beef,1002521,Box O Wine,"Instead of roasting the red peppers, I just cu...",[https://img.sndimg.com/food/image/upload/w_55...,Main Dish,"[Meat, Weeknight, < 60 Mins, Easy]","[1, 1, 2, 1, 1, 1, 2, 1, 1⁄4, 1⁄4, 1⁄4, 2, 1⁄8]","[lean ground beef, garlic cloves, yellow onion...",NaN,338.8,15.1,5.1,73.7,80.0,23.9,1.3,1.5,25.1,4.0,"[In a large skillet, cook ground beef over med...",https://www.food.com/recipe/Spanish-Rice-With-...,"[1, 1, 2, 1, 1, 1, 2, 1, 1⁄4, 1⁄4, 1⁄4, 2, 1⁄8]","[lb lean ground beef, tablespoon vegetable oil...",2009,8,17,3,50,NaN,-1_pepper_onion_garlic,main dishMeat Weeknight < 60 Mins Easylb lean ...,147,"[0.004986213985830545, -0.0030652128625661135,...",84.70,3.78,1.27,18.42,20.00,5.98,0.32,0.38,6.28,1.0,0.888753
49605,247296.0,Stuffed Burger,265694,Brisket in Roses,"I was reading a recipe yesterday on zaar, and ...",[https://img.sndimg.com/food/image/upload/w_55...,Meat,"[< 30 Mins, Beginner Cook, Easy]","[2, 5, 2, 1, None, None, 8, None, None, None, ..

In [25]:
print(f"Runtime: {round(end_time-start_time,2)} seconds.")

Runtime: 65.13 seconds.


In [26]:
top_5_similar_recipes.iloc[0]['CleanRecipeTextData']

'meat< 15 Mins Beginner Cook Easylb ground beef (or chuck or sirloin) cup dried cilantro (or a handful of fresh chopped cilantro) tablespoons minced garlic cup honey tablespoons soy sauce tablespoons ginger paste fresh ground pepper (peppercorn medley) buns (one for each patty) fresh minced onions (optional) or dried parsley (optional) salt (optional) canned black beans (small can, drained and rinsed) (optional) prepared spaghetti sauce (optional) or pizza sauce (optional)mix ingredi togeth except ground pepper form patti prefer size thickness grill burger prefer done rare welldone crack fresh ground pepper burger grill desired salt burger tast top chees last minut grill melt burger burger alreadi soy sauce find salt unnecessary place burger favorit bun add toppings dig in options add option ingredi mix grilling fresh minc onion add alot flavor everyon like onions ad black bean easi way bulk mix add great fiber nutrition might wish tri half first time unsur like them burger verg meatlo

Just from a first glance we realize that both of these recipes are burger recipes.

**The user input recipe:**
<img src="https://images.themodernproper.com/billowy-turkey/production/posts/2019/Classic-Cheeseburger-With-Secret-Sauce-13.jpg?w=1800&q=82&fm=jpg&fit=crop&dm=1606105565&s=28009d0ef053def9f7f450310ebce714" alt="Beautiful Sunset" width="400" height="300">


**The top-recommended recipe:**

![Burger2](https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/21/88/75/pic5KXgef.jpg)

So far so good!! Let's take a look at the text data:

In [27]:
# Find exact matches

String1 = user_input
String2 = top_5_similar_recipes.iloc[0]['CleanRecipeTextData']
find_common_words(String1, String2)

{'a',
 'and',
 'beef',
 'bun',
 'burger',
 'fresh',
 'ground',
 'in',
 'melt',
 'of',
 'onions',
 'pizza',
 'sauce'}

#### Similar Terms

Aside from the the exact matches that we see above, here are some words and phrases in the two strings that are semantically similar (where `String 1` = user_input, and `String 2` = the first recommneded similar recipe):

- "ground beef" (both strings)
- "ginger paste" (String 1) vs. "tablespoons ginger paste" (String 2)
- "soy sauce" (both strings)
- "fresh ground pepper" (String 1) vs. "fresh ground pepper (peppercorn medley)" (String 2)
- "cilantro" (String 1) vs. "dried cilantro (or a handful of fresh chopped cilantro)" (String 2)
- "buns" (both strings)
- "onions" (String 1) vs. "fresh minced onions (optional)" (String 2)
- "spaghetti sauce" (String 2) vs. "marinara sauce" (String 1)
- "grill burger" (both strings)

#### Categories

We can also see that most of the recommended recipes fall under the categories 'Meat' or "Main Dish' (containing meaty contents) (in `RecipeCategory`). The BerTopic categories don't seem much helpful here: even though they're all in the topic `-1_pepper_onion_garlic`, this topic (starting with `-1` is the topic that BerTopic throws in it things that it cannot properly categorize). This might have to do with the fact that the topics that we have imported from BerTopic are reduced from over 2,400 to 300, which means that many of the more nuanced categoris were tossed into `-1_pepper_onion_garlic` (see the notebook on topic modeling). We might be able to mitigate the issue if we use the original 2,400 topics; I won't pursue this here, however.

Let's look at more examples:

In [28]:
user_input = ("Grilled Salmon with Asparagus: A delicious and healthy dish featuring grilled salmon fillets served with roasted asparagus."
             )

top_5_similar_recipes = most_similar_recipes(user_input, recipes)
top_5_similar_recipes

,RecipeId,Name,AuthorId,AuthorName,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeInstructions,url,ingred_quants,ingred_items,YearPublished,MonthPublished,DayPublished,HourPublished,TotalMinutes,CorrectAggregatedRating,BerTopics,CleanRecipeTextData,num_tokens,ada_embeddings,Caloriesfor1,FatContentfor1,SaturatedFatContentfor1,CholesterolContentfor1,SodiumContentfor1,CarbohydrateContentfor1,FiberContentfor1,SugarContentfor1,ProteinContentfor1,RecipeServingsfor1,SimilarityRate
74737,368558.0,Grilled Salmon and Asparagus Salad,464080,januarybride,A fun recipe to make on the grill. Maple syru...,[https://img.sndimg.com/food/image/upload/w_55...,Sauces/Condiments,"[One Dish Meal, Vegetable, Spring, Summer, < 3...","[1⁄3, 2, 1, 1, 1, 4, 1, 1, None]","[Dijon mustard, olive oil, salmon fillet, aspa...",3.0,283.4,9.2,1.6,112.1,232.0,24.1,3.5,11.7,27.5,4.0,"[Heat gas or charcoal grill., In small bowl, m...",https://www.food.com/recipe/Grilled-Salmon-and...,"[1⁄3, 2, 1, 1, 1, 4, 1, 1, ]","[cup maple-flavored syrup, tablespoons Dijon m...",2009,4,29,14,30,5.00,0_asparagus_salmon_fillet,saucescondiOne Dish Meal Vegetable Spring Summ...,146,"[0.0140702398493886, -0.0013422268675640225, -...",70.85,2.30,0.40,28.02,58.00,6.02,0.88,2.92,6.88,1.0,0.884548
78279,385534.0,Grilled Salmon and Asparagus With Balsamic Butter,865936,threeovens,A balsamic reduction is mixed with a bit of bu...,[https://img.sndimg.com/food/image/upload/w_55...,Vegetables,"[Very Low Carbs, < 30 Mins]","[1⁄2, 1, 1, None, 1, 3, 4]","[honey, butter, asparagus, extra virgin olive ...",NaN,587.8,27.2,5.8,153.9,287.5,14.4,2.3,10.6,68.1,4.0,[Preheat outdoor grill or indoor grill pan to ...,https://www.food.com/recipe/Grilled-Salmon-and...,"[1⁄2, 1, 1, , 1, 3, 4]","[cup balsamic vinegar, tablespoon honey, table...",2009,8,17,2,30,NaN,-1_pepper_onion_garlic,vegetVery Low Carbs < 30 Minscup balsamic vine...,95,"[0.001755754929035902, -0.008465186692774296, ...",146.95,6.80,1.45,38.48,71.88,3.60,0.57,2.65,17.02,1.0,0.880817
83351,415538.0,Salmon With Asparagus and Chive Butter Sauce,653438,mama smurf,Found in the Best Simple Recipes (America's T...,[https://img.sndimg.com/food/image/upload/w_55...,Vegetables,"[Very Low Carbs, < 30 Mins, Easy]","[1, 1, None, 4, None, 1⁄2, 3, 2, 1]","[asparagus, water, salt, salmon fillets, peppe...",2.0,326.8,14.8,6.5,110.4,133.3,8.4,3.6,1.8,36.8,4.0,[Lay asparagus in single layer on bottom of a ...,https://www.food.com/recipe/Salmon-With-Aspara...,"[1, 1, , 4, , 1⁄2, 3, 2, 1]","[lb thick asparagus, trimmed, cup water, salt,...",2010,3,5,16,25,5.00,0_asparagus_salmon_fillet,vegetVery Low Carbs < 30 Mins Easylb thick asp...,109,"[0.0090714106336236, -0.008792812936007977, 0....",81.70,3.70,1.62,27.60,33.33,2.10,0.90,0.45,9.20,1.0,0.878275
20639,113817.0,Salmon and Asparagus in Foil,133174,PaulaG,This is a quick and easy dinner. Pop the pack...,[https://img.sndimg.com/food/image/upload/w_55...,Main Dish,"[Vegetable, Canadian, Scandinavian, European, ...","[4, 4, 1, 1, None, None, None]","[salmon fillets, asparagus, lemon, fresh groun...",109.0,207.3,6.5,1.2,64.5,121.4,6.0,2.7,1.8,31.6,4.0,"[Preheat oven to 450 degrees., Snap the ends o...",https://www.food.com/recipe/Salmon-and-Asparag...,"[4, 4, 1, 1, , , ]","[sheets aluminum foil, wrap (12-inch by 18-inc...",2005,3,21,16,35,4.65,0_asparagus_salmon_fillet,main dishVegetable Canadian Scandinavian Europ...,118,"[0.02600717358291149, -0.005210068076848984, -...",51.82,1.62,0.30,16.12,30.35,1.50,0.68,0.45,7.90,1.0,0.873924
62299,305328.0,Grilled Garlic Asparagus and Salmon,262272,cookee monster,Make and share this Grilled Garlic Asparagus a...,[https://img.sndimg.com/food/image/upload/w_55...,Pasta,"[Vegetable, Very Low Carbs, < 30 Mins, Easy]","[5, 8, 1, 1⁄2, 4]","[olive oil, garlic cloves, salmo

In [29]:
top_5_similar_recipes['CleanRecipeTextData'].values

array(['saucescondiOne Dish Meal Vegetable Spring Summer < 30 Mins Beginner Cook Easycup maple-flavored syrup tablespoons Dijon mustard tablespoon olive oil lb salmon fillet (1/2 inch thick) lb fresh asparagus spear cups baby greens cup shredded carrot (about 1 1/2 medium) hard-cooked egg, cut into 8 wedges fresh ground black pepperheat ga charcoal grill small bowl mix mapledijon dress ingredi wire whisk cut salmon crosswis 4 pieces brush salmon 1 tablespoon dressing larg bowl toss asparagu 1 tablespoon dressing place asparagu grill basket grill “wok” grill heated place asparagu salmon skin side down ga grill medium heat charcoal grill medium coals cover grill cook asparagu 7 10 minut salmon 10 15 minutes shake grill basket turn asparagu occasionally asparagu crisptend salmon flake easili fork slide pancak turner salmon skin remov piec skin 4 plates divid salad greens carrot egg top asparagu salmon sprinkl pepper serv remain dressing',
       'vegetVery Low Carbs < 30 Minscup balsamic 

Yet another one:

In [30]:
user_input = ("Caprese Salad: A simple and refreshing salad made with fresh tomatoes, mozzarella cheese, basil leaves, and a drizzle of balsamic glaze or olive oil."
             )

top_5_similar_recipes = most_similar_recipes(user_input, recipes)
top_5_similar_recipes

,RecipeId,Name,AuthorId,AuthorName,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeInstructions,url,ingred_quants,ingred_items,YearPublished,MonthPublished,DayPublished,HourPublished,TotalMinutes,CorrectAggregatedRating,BerTopics,CleanRecipeTextData,num_tokens,ada_embeddings,Caloriesfor1,FatContentfor1,SaturatedFatContentfor1,CholesterolContentfor1,SodiumContentfor1,CarbohydrateContentfor1,FiberContentfor1,SugarContentfor1,ProteinContentfor1,RecipeServingsfor1,SimilarityRate
24681,132367.0,Tomato and Fresh Mozzarella Salad With Arugula...,159145,Simply Chris,"Really simple, easy to prepare salad. Just a ...",[https://img.sndimg.com/food/image/upload/w_55...,Vegetables,"[< 15 Mins, Easy]","[1, 4, 3, 12, None, None, 1, 2, 1⁄4]","[tomatoes, fresh mozzarella balls, yellow bell...",2.0,379.1,27.0,9.4,44.8,367.7,21.8,3.5,2.5,16.2,2.0,"[Whisk balsamic vinegar, olive oil and dried b...",https://www.food.com/recipe/Tomato-and-Fresh-M...,"[1, 4, 3, 12, , , 1, 2, 1⁄4]","[medium tomatoes, thinly sliced, ounces fresh ...",2005,8,4,21,10,4.50,0_asparagus_salmon_fillet,"veget< 15 Mins Easymedium tomatoes, thinly sli...",71,"[0.005124865099787712, -0.0088651767000556, 0....",189.55,13.50,4.70,22.40,183.85,10.90,1.75,1.25,8.10,1.0,0.881426
73790,363554.0,Fresh Mozzarella &amp; Tomato Salad With Balsa...,204124,cajunhippiegirl,A great summer recipe when the tomatoes are pl...,[https://img.sndimg.com/food/image/upload/w_55...,Quick and Easy,[],"[8, 3, 3, 2, 1⁄4, 3, 1⁄2, 1⁄4, None]","[fresh mozzarella balls, tomatoes, basil, oliv...",6.0,317.6,26.4,9.4,44.9,516.3,6.9,1.1,4.8,13.5,4.0,"[Mix Vinegar, mustard, salt and pepper togethe...",https://www.food.com/recipe/Fresh-Mozzarella-&...,"[8, 3, 3, 2, 1⁄4, 3, 1⁄2, 1⁄4, ]","[ounces fresh mozzarella balls, sliced, cups f...",2009,3,30,17,10,4.83,0_asparagus_salmon_fillet,"quick easiounces fresh mozzarella balls, slice...",61,"[0.0018687495030462742, -0.007501646876335144,...",79.40,6.60,2.35,11.22,129.07,1.72,0.28,1.20,3.38,1.0,0.880437
28865,152016.0,"Wisconsin Fresh Mozzarella, Tomato and Basil S...",189643,Kaarin,This is a wonderful salad! Fresh mozzarella ma...,[https://img.sndimg.com/food/image/upload/w_55...,Dairy,"[Vegetable, Summer, Savory, < 15 Mins, Easy]","[2, 8, 2, 1, 2, 1⁄4, 1⁄8, 1]","[tomatoes, fresh mozzarella cheese, fresh basi...",2.0,309.8,26.4,9.4,44.8,433.5,5.7,1.3,3.0,13.6,4.0,[In a small bowl whisk together the dressing i...,https://www.food.com/recipe/Wisconsin-Fresh-Mo...,"[2, 8, 2, 1, 2, 1⁄4, 1⁄8, 1]","[large ripe tomatoes (vine-ripened are best), ...",2006,1,18,12,10,5.00,0_asparagus_salmon_fillet,dairiVegetable Summer Savory < 15 Mins Easylar...,82,"[0.020316921174526215, -0.006120691541582346, ...",77.45,6.60,2.35,11.20,108.38,1.42,0.32,0.75,3.40,1.0,0.878312
47811,239050.0,Tomato &amp; Mozzarella Caprese - Official Rec...,531768,Starfire aka Wendy,Make and share this Tomato &amp; Mozzarella Ca...,[https://img.sndimg.com/food/image/upload/w_55...,Dairy,"[Vegetable, High In..., < 30 Mins]","[8, 2, 8, 12, None, None, None, 2]","[tomatoes, fresh basil leaves, fresh mozzarell...",4.0,329.7,25.9,12.2,67.3,538.2,4.8,0.5,3.1,19.3,4.0,"[Arrange sliced tomatoes on a large platter., ...",https://www.food.com/recipe/Tomato-&amp;-Mozza...,"[8, 2, 8, 12, , , , 2]","[slices of vine-ripened tomatoes, tablespoons ...",2007,7,6,23,25,4.75,0_asparagus_salmon_fillet,dairiVegetable High In... < 30 Minsslices of v...,72,"[0.006005996372550726, -0.010141660459339619, ...",82.42,6.48,3.05,16.82,134.55,1.20,0.12,0.78,4.82,1.0,0.874512
108510,538981.0,Caprese Stuffed Avocado,400708,Bonnie G 2,I love Caprese salads and avocado so when I sa...,[https://img.sndimg.com/food/image/upload/w_55...,Fruit,"[Vegetable, < 15 Mins, Easy]","[1⁄2, 4, 2, 1, 1⁄4, None, 2, None, 2, 2]","[cherry tomatoes, mozz

In [31]:
top_5_similar_recipes['CleanRecipeTextData'].values

array(['veget< 15 Mins Easymedium tomatoes, thinly sliced ounces fresh mozzarella balls, sliced baby yellow bell peppers, sliced large arugula leaves, coarsely chopped sea salt fresh fresh coarse ground black pepper teaspoon balsamic vinegar tablespoons extra virgin olive oil teaspoon dried basilwhisk balsam vinegar oliv oil dri basil togeth small bowl arrang altern slice tomato chees bell pepper slice platter scatter chop arugula tomatoes drizzl oliv oil mixtur salad season salt pepper',
       'quick easiounces fresh mozzarella balls, sliced cups fresh baby greens tomatoes, sliced tablespoons fresh basil, chpped cup olive oil tablespoons balsamic vinegar teaspoon Dijon mustard teaspoon salt (or to taste) fresh ground peppermix vinegar mustard salt pepper together add oil whisk well blended spread green plate bowl cover tomato slices arrang chees slice top tomatoes drizzl dress salad sprinkl fresh basil',
       'dairiVegetable Summer Savory < 15 Mins Easylarge ripe tomatoes (vine-rip

## A Nutrition-Based Recommender System <a class ='author' id='nutr'>

We now prropose a different recommender system that operates on nutritional factors associated with recipes (such as protein content and fat content). 

Many of the recipes in the dataset have their ingredient ammounts appropriate for *more than* one person. As a result, the nutritional factors of the recipe are naturally multiplied by this factor. Since one of our recommender systems later will rely on the the numbers assigned to these nutritional factors, we need to settle the issue here and make sure all of these numbers are approporated for one person only.

**NOTE:** for simplicity and efgiciency reasons, we won't be implementing this recommender system in our Streamlit app.


In [ ]:
nutritional_cols = ['Calories', 'FatContent', 'SaturatedFatContent', 'CholesterolContent',
                    'SodiumContent', 'CarbohydrateContent', 'FiberContent', 'SugarContent',
                    'ProteinContent', 'RecipeServings']

In [ ]:
# Apply the transformation to each nutritional column separately
# We don't touch the original nutriotional columns because later on we want to retrieve them as they are, and for
# how many people they are

for col in nutritional_cols:
    recipes[col+'for1'] = recipes[col].div(recipes['RecipeServings']).round(2)

# Now, the nutritional values have been divided by 'RecipeServings' and rounded to 2 decimal places

We can implement the recommender system using a pipeline. Here's an example of how you can use scikit-learn's pipeline to build the recommender system:

Let's first try Euclidean distance as our measure:

In [35]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

df = recipes.copy()

# Select the columns used for similarity calculation
selected_columns = ['Calories', 'FatContent', 'SaturatedFatContent', 'CholesterolContent',
                    'SodiumContent', 'CarbohydrateContent', 'FiberContent', 'SugarContent',
                    'ProteinContent']
nutritional_cols = [item + 'for1' for item in nutritional_cols]

# Define the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Step 1: Scale the selected features
    ('knn', NearestNeighbors(metric='euclidean'))  # Step 2: Apply the KNN model with cosine similarity metric
])

# Fit the pipeline
scaled_features = pipeline['scaler'].fit_transform(df[selected_columns].values)
pipeline['knn'].fit(scaled_features)

# Function to get similar recipes
def get_similar_recipes(recipe_id, top_n=20):
    recipe_index = df[df['RecipeId'] == recipe_id].index[0]
    recipe_features = df.loc[recipe_index, selected_columns].values.reshape(1, -1)
    recipe_features_scaled = pipeline['scaler'].transform(recipe_features)
    _, top_indices = pipeline['knn'].kneighbors(recipe_features_scaled, n_neighbors=top_n+1)
    top_indices = top_indices.flatten()[1:]
    top_recipe_ids = df.loc[top_indices, 'RecipeId'].values
    top_recipes = df[df['RecipeId'].isin(top_recipe_ids)]
    return top_recipes

# Example usage:
recipe_id = 132367  # Replace with the actual RecipeId you want recommendations for
top_similar_recipes = get_similar_recipes(recipe_id, top_n=20)
pd.concat([df[df['RecipeId'] == recipe_id],get_similar_recipes(recipe_id, top_n=20)])[selected_columns][:5]

,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent
24681,379.1,27.0,9.4,44.8,367.7,21.8,3.5,2.5,16.2
2424,354.7,23.3,8.9,43.7,272.0,25.0,2.6,1.6,11.4
5997,336.4,24.3,9.9,45.5,534.8,17.4,2.9,5.5,13.3
9272,345.9,24.3,9.7,48.5,513.7,16.5,2.7,5.8,16.8
12855,361.1,25.1,9.1,44.2,622.3,19.4,2.4,4.7,16.1


Now let's try cosine similarity as our measure:

In [36]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

df = recipes.copy()

# Select the columns used for similarity calculation
selected_columns = ['Calories', 'FatContent', 'SaturatedFatContent', 'CholesterolContent',
                    'SodiumContent', 'CarbohydrateContent', 'FiberContent', 'SugarContent',
                    'ProteinContent']

# Define the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Step 1: Scale the selected features
    ('knn', NearestNeighbors(metric='cosine'))  # Step 2: Apply the KNN model with cosine similarity metric
])

# Fit the pipeline
scaled_features = pipeline['scaler'].fit_transform(df[selected_columns].values)
pipeline['knn'].fit(scaled_features)

# Function to get similar recipes
def get_similar_recipes(recipe_id, top_n=20):
    recipe_index = df[df['RecipeId'] == recipe_id].index[0]
    recipe_features = df.loc[recipe_index, selected_columns].values.reshape(1, -1)
    recipe_features_scaled = pipeline['scaler'].transform(recipe_features)
    _, top_indices = pipeline['knn'].kneighbors(recipe_features_scaled, n_neighbors=top_n+1)
    top_indices = top_indices.flatten()[1:]
    top_recipe_ids = df.loc[top_indices, 'RecipeId'].values
    top_recipes = df[df['RecipeId'].isin(top_recipe_ids)]
    return top_recipes

# Example usage:
recipe_id = 132367  # Replace with the actual RecipeId you want recommendations for
top_similar_recipes = get_similar_recipes(recipe_id, top_n=20)
pd.concat([df[df['RecipeId'] == recipe_id],get_similar_recipes(recipe_id, top_n=20)])[selected_columns][:5]

,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent
24681,379.1,27.0,9.4,44.8,367.7,21.8,3.5,2.5,16.2
282,494.3,42.4,13.4,38.6,499.5,14.1,3.3,4.3,17.9
5997,336.4,24.3,9.9,45.5,534.8,17.4,2.9,5.5,13.3
9272,345.9,24.3,9.7,48.5,513.7,16.5,2.7,5.8,16.8
12457,399.8,36.0,8.3,11.4,783.1,7.2,3.9,1.1,19.5


Just by taking a glance, it seems that Euclidean distnace is doing a better job at finding similar recipes accross various nutritional factors. This can be measured more reigorously but since we don't be implementing this particular recommender system in our streamlit, I won't pursue this here.

## Ensembling the Recommender Systems <a class ='author' id='ensemble'>

We could in principle deploy both of our recommender systems into one recommender system with two layers. An example use case woud be a user querying the reommender system based on their favorite ingredients, themse and other keywords, and once receive a list of similar recipes, searcing through the dataset to find nutritionally similar ones to, say, the top 3 semantically similar recipes.

In [37]:
from heapq import nlargest

def most_similar_recipes(user_input, df, n=len(df)):
    """
    This function takes an input and sorts out the dataset based on cosine similarities to the user input.
    """
    # Step 1: Get the embeddings for the user_input
    input_embeddings = get_embedding(user_input)

    # Step 2: Calculate the similarity between user_input and each row in the DataFrame
    similarities = []

    for index, row in df.iterrows():
        cur_embeddings = row['ada_embeddings']
        similarity = cosine_similarity(np.array(input_embeddings).reshape(1, -1), np.array(cur_embeddings).reshape(1, -1))[0][0]
        similarities.append(similarity)

    recipes['SimilarityRate'] = similarities
    
    # Step 3: Return the DataFrame rows with sorted using SimilarityRate, from higher to lower
    return df.sort_values(['SimilarityRate'],ascending=False)

Let's prompt the semantic-based recommender system with some recipe instructions.

In [38]:
start_time = time.time()

user_input = (
"Easy Ground Beef Recipes with Few Ingredients Thai Basil Beef (cover photo). A speedy, savory Thai beef basil stir fry that’s\
just a bit spicy and really hits the spot. Gingery Ground Beef (Soboro Donburi). Five ingredients, a few minutes and a hot \
skillet, and you’ll be digging into a delicious soboro donburi, a gingery ground beef that reminds us that the best Japanese \
recipes are often the simplest. Crock-Pot Taco Meat. Just three-ingredients and the use of our trusty slow-cooker make this \
Crock-Pot beef taco meat recipe an easy win. Taco Pizza. Two classic family favorites, pizza and tacos, come together in this\
super easy weeknight meal that is destined to become a favorite in your home. Ground Beef Burger and Sandwich Recipes Classic\
Cheeseburger with Secret Sauce. A perfectly soft bun, quality, juicy ground beef that is seasoned well, plenty of cheese,\
caramelized onions and a really good secret sauce are the key to this truly classic cheeseburger recipe. Patty Melt. As diner\
sandwiches go, the melty, meaty, oniony perfection that is the patty melt reigns supreme—if you know, you know. And if you don’t\
know, well, welcome to the club. Meatball Sub Sandwich. Big juicy, tender meatballs simmered in marinara sauce (store-bought\
or homemade—either works!) stuffed into garlicky hoagie rolls, topped with mozzarella and broiled to melty, bubbly perfection.\
Beef Sliders. These juicy ground beef sliders—slathered with a homemade bright and lemony sun-dried tomato mayo, spicy pickles,\
and fresh arugula—prove that bigger doesn’t always equal better. Low-Carb Bacon Burger with Guacamole. You’ll forget all about\
the missing bun while devouring this juicy, keto-friendly bacon guacamole burger loaded with veggies and jalapeno aioli!"
)

top_similar_recipes = most_similar_recipes(user_input, recipes)

end_time = time.time()

top_similar_recipes

,RecipeId,Name,AuthorId,AuthorName,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeInstructions,url,ingred_quants,ingred_items,YearPublished,MonthPublished,DayPublished,HourPublished,TotalMinutes,CorrectAggregatedRating,BerTopics,CleanRecipeTextData,num_tokens,ada_embeddings,Caloriesfor1,FatContentfor1,SaturatedFatContentfor1,CholesterolContentfor1,SodiumContentfor1,CarbohydrateContentfor1,FiberContentfor1,SugarContentfor1,ProteinContentfor1,RecipeServingsfor1,SimilarityRate
43303,218875.0,Garlic Lover's Burger,178711,Buckeye Karen,"If you're like me, you just can't get enough o...",[https://img.sndimg.com/food/image/upload/w_55...,Meat,"[< 15 Mins, Beginner Cook, Easy]","[1, 1⁄8, 3, 1⁄4, 3, 1 1⁄2, None, 4, None, None...","[ground beef, dried cilantro, garlic, honey, s...",1.0,449.9,19.0,7.1,77.1,1039.0,42.4,1.3,20.5,27.3,4.0,[Mix all ingredients together (except ground p...,https://www.food.com/recipe/Garlic-Lover's-Bur...,"[1, 1⁄8, 3, 1⁄4, 3, 1 1⁄2, , 4, , , , ]","[lb ground beef (or chuck or sirloin), cup dri...",2007,3,26,20,15,5.00,-1_pepper_onion_garlic,meat< 15 Mins Beginner Cook Easylb ground beef...,170,"[-0.007803387939929962, 0.002627050504088402, ...",112.48,4.75,1.78,19.27,259.75,10.60,0.32,5.12,6.82,1.0,0.893553
107680,536579.0,Korean Ground Beef and Rice Bowls,972011,Yearbook Mom,Make and share this Korean Ground Beef and Ric...,[https://img.sndimg.com/food/image/upload/w_55...,Quick and Easy,[Easy],"[1, 3, 1⁄4, 1⁄4, 2, 1⁄4, 1⁄4, 1⁄4, 2, 1⁄4, 1]","[lean ground beef, garlic cloves, brown sugar,...",NaN,407.5,13.8,5.0,73.7,611.4,42.9,0.7,13.8,26.0,4.0,[1. In a large skillet cook the ground beef an...,https://www.food.com/recipe/Korean-Ground-Beef...,"[1, 3, 1⁄4, 1⁄4, 2, 1⁄4, 1⁄4, 1⁄4, 2, 1⁄4, 1]","[lb lean ground beef (90% lean), garlic cloves...",2018,7,30,18,20,NaN,-1_pepper_onion_garlic,quick easiEasylb lean ground beef (90% lean) g...,89,"[-0.003680585417896509, -0.00722745293751359, ...",101.88,3.45,1.25,18.42,152.85,10.72,0.18,3.45,6.50,1.0,0.890098
104057,525697.0,Thai Basil Beef,403989,Papagayita,"Delicious, adaptable recipe that's great for u...",[https://img.sndimg.com/food/image/upload/w_55...,Meat,"[Thai, Asian, Weeknight, < 30 Mins, Easy]","[1, 2, 3 -4, 1, 1, 1 -2, 1, 1, 2, 1⁄4, 1, 1⁄2,...","[ground beef, garlic cloves, onion, bell peppe...",1.0,302.6,19.7,7.1,77.6,929.4,7.6,1.3,3.7,23.1,4.0,[Heat a large frying pan over medium high heat...,https://www.food.com/recipe/Thai-Basil-Beef-52...,"[1, 2, 3 -4, 1, 1, 1 -2, 1, 1, 2, 1⁄4, 1, 1⁄2, ]","[lb ground beef, teaspoons sesame oil, garlic ...",2016,1,24,15,25,5.00,-1_pepper_onion_garlic,meatThai Asian Weeknight < 30 Mins Easylb grou...,139,"[-0.004620139487087727, -0.007158639840781689,...",75.65,4.92,1.78,19.40,232.35,1.90,0.32,0.92,5.78,1.0,0.888877
78319,385719.0,Spanish Rice With Beef,1002521,Box O Wine,"Instead of roasting the red peppers, I just cu...",[https://img.sndimg.com/food/image/upload/w_55...,Main Dish,"[Meat, Weeknight, < 60 Mins, Easy]","[1, 1, 2, 1, 1, 1, 2, 1, 1⁄4, 1⁄4, 1⁄4, 2, 1⁄8]","[lean ground beef, garlic cloves, yellow onion...",NaN,338.8,15.1,5.1,73.7,80.0,23.9,1.3,1.5,25.1,4.0,"[In a large skillet, cook ground beef over med...",https://www.food.com/recipe/Spanish-Rice-With-...,"[1, 1, 2, 1, 1, 1, 2, 1, 1⁄4, 1⁄4, 1⁄4, 2, 1⁄8]","[lb lean ground beef, tablespoon vegetable oil...",2009,8,17,3,50,NaN,-1_pepper_onion_garlic,main dishMeat Weeknight < 60 Mins Easylb lean ...,147,"[0.004986213985830545, -0.0030652128625661135,...",84.70,3.78,1.27,18.42,20.00,5.98,0.32,0.38,6.28,1.0,0.888754
49605,247296.0,Stuffed Burger,265694,Brisket in Roses,"I was reading a recipe yesterday on zaar, and ...",[https://img.sndimg.com/food/image/upload/w_55...,Meat,"[< 30 Mins, Beginner Cook, Easy]","[2, 5, 2, 1, None, None, 8, None, None, None, 

In [39]:
print(f"Runtime: {round(end_time-start_time,2)} seconds.")

Runtime: 65.26 seconds.


In [41]:
top_similar_recipes.iloc[1][selected_columns]

Calories               407.5
FatContent              13.8
SaturatedFatContent      5.0
CholesterolContent      73.7
SodiumContent          611.4
CarbohydrateContent     42.9
FiberContent             0.7
SugarContent            13.8
ProteinContent          26.0
Name: 107680, dtype: object

We now apply the nutritional filter to the most similar recipes:

In [42]:
recipe_id = 218875.0  # Replace with the actual RecipeId you want recommendations for
top_similar_recipes = get_similar_recipes(recipe_id, top_n=20)

# For comparsion purposes, let's concatennate the recipe with recipe_id = 218875.0 at the top of our list
# The most similar item in terms of nutritional factors is the second row in the dataframe

top_similar_recipes = pd.concat([df[df['RecipeId'] == recipe_id],get_similar_recipes(recipe_id, top_n=20)])

top_similar_recipes

,RecipeId,Name,AuthorId,AuthorName,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeInstructions,url,ingred_quants,ingred_items,YearPublished,MonthPublished,DayPublished,HourPublished,TotalMinutes,CorrectAggregatedRating,BerTopics,CleanRecipeTextData,num_tokens,ada_embeddings,Caloriesfor1,FatContentfor1,SaturatedFatContentfor1,CholesterolContentfor1,SodiumContentfor1,CarbohydrateContentfor1,FiberContentfor1,SugarContentfor1,ProteinContentfor1,RecipeServingsfor1,SimilarityRate
43303,218875.0,Garlic Lover's Burger,178711,Buckeye Karen,"If you're like me, you just can't get enough o...",[https://img.sndimg.com/food/image/upload/w_55...,Meat,"[< 15 Mins, Beginner Cook, Easy]","[1, 1⁄8, 3, 1⁄4, 3, 1 1⁄2, None, 4, None, None...","[ground beef, dried cilantro, garlic, honey, s...",1.0,449.9,19.0,7.1,77.1,1039.0,42.4,1.3,20.5,27.3,4.0,[Mix all ingredients together (except ground p...,https://www.food.com/recipe/Garlic-Lover's-Bur...,"[1, 1⁄8, 3, 1⁄4, 3, 1 1⁄2, , 4, , , , ]","[lb ground beef (or chuck or sirloin), cup dri...",2007,3,26,20,15,5.00,-1_pepper_onion_garlic,meat< 15 Mins Beginner Cook Easylb ground beef...,170,"[-0.007803387939929962, 0.002627050504088402, ...",112.48,4.75,1.78,19.27,259.75,10.60,0.32,5.12,6.82,1.0,0.772547
7540,43986.0,Country Sloppy Joes,50053,dayla,I found this recipe on the back of a Bear Cree...,[https://img.sndimg.com/food/image/upload/w_55...,Lunch/Snacks,"[Steak, Meat, Sweet, Brunch, < 30 Mins, Stove ...","[1 1⁄2, None, 2, 1, 1, 6]","[ground beef, brown sugar, water, ketchup]",32.0,420.1,19.0,7.1,77.1,728.9,35.8,1.0,16.3,25.9,6.0,"[In a large skillet, brown ground beef over me...",https://www.food.com/recipe/Country-Sloppy-Joe...,"[1 1⁄2, , 2, 1, 1, 6]","[lbs ground beef, onion soup mix, 1 packet, ta...",2002,10,25,22,20,4.63,-1_pepper_onion_garlic,lunchsnackSteak Meat Sweet Brunch < 30 Mins St...,61,"[-0.006306047551333904, -0.0021221612114459276...",70.02,3.17,1.18,12.85,121.48,5.97,0.17,2.72,4.32,1.0,0.763127
11509,69202.0,Smoky Mountain Chicken,10404,Diana Adcock,Make and share this Smoky Mountain Chicken rec...,[https://img.sndimg.com/food/image/upload/w_55...,Chicken,"[Chicken, Pork, Poultry, Cheese, Onions, Veget...","[4, 1, 1, 1, 18, 8, 4, 1⁄4, 1⁄2]","[boneless skinless chicken breasts, ground bla...",52.0,533.2,19.2,8.4,111.3,1614.0,53.5,1.5,37.5,34.3,4.0,"[Preheat oven to 350 degrees., Lightly grease ...",https://www.food.com/recipe/Smoky-Mountain-Chi...,"[4, 1, 1, 1, 18, 8, 4, 1⁄4, 1⁄2]","[boneless skinless chicken breasts, teaspoon g...",2003,8,20,20,43,4.63,-1_pepper_onion_garlic,chickenChicken Pork Poultry Cheese Onions Vege...,110,"[0.010160657577216625, -0.01195566263049841, -...",133.30,4.80,2.10,27.82,403.50,13.38,0.38,9.38,8.57,1.0,0.781087
13665,83717.0,Barbecued Meatballs,85018,mickie49,Make and share this Barbecued Meatballs recipe...,[https://img.sndimg.com/food/image/upload/w_55...,Meat,[< 4 Hours],"[1, 1⁄2, 1, 1⁄2, 1, 1, 1, 3, 1, 1⁄4]","[ground beef, salt, evaporated milk, ketchup, ...",3.0,457.3,20.4,8.3,86.2,1504.2,42.5,1.5,26.5,26.5,4.0,"[Combine first four ingredients., Shape into m...",https://www.food.com/recipe/Barbecued-Meatball...,"[1, 1⁄2, 1, 1⁄2, 1, 1, 1, 3, 1, 1⁄4]","[lb ground beef (chuck works well), cup plain ...",2004,2,12,19,70,5.00,-1_pepper_onion_garlic,meat< 4 Hourslb ground beef (chuck works well)...,74,"[-0.009528647176921368, 0.0007873811991885304,...",114.32,5.10,2.08,21.55,376.05,10.62,0.38,6.62,6.62,1.0,0.756705
14453,87563.0,Sloppy Joes,124343,JudaThann,I've been making this recipe for over 30 years...,[https://img.sndimg.com/food/image/upload/w_55...,Lunch/Snacks,"[Meat, Kid Friendly, < 60 Mins, Beginner Cook,...","[1, 1⁄2, 1⁄2, 1, 1⁄4, 2, 2, 1, 4]","[lean ground beef, green bell pepper, onion, k...",8.0,398.9,13.5,5.1,73.7,966.5,41.4,1.7,19

In [45]:
top_similar_recipes[selected_columns]

,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent
43303,449.9,19.0,7.1,77.1,1039.0,42.4,1.3,20.5,27.3
7540,420.1,19.0,7.1,77.1,728.9,35.8,1.0,16.3,25.9
11509,533.2,19.2,8.4,111.3,1614.0,53.5,1.5,37.5,34.3
13665,457.3,20.4,8.3,86.2,1504.2,42.5,1.5,26.5,26.5
14453,398.9,13.5,5.1,73.7,966.5,41.4,1.7,19.8,28.2
16498,394.3,17.0,5.3,71.4,1073.3,34.6,0.9,18.2,25.8
27701,486.2,24.9,8.8,104.3,1581.8,37.1,0.2,33.4,30.2
36111,445.4,15.9,5.3,75.5,1109.9,46.9,1.6,17.1,27.9
38825,395.2,13.6,5.1,73.7,960.5,40.1,1.6,18.4,28.2
40631,426.3,17.6,8.3,96.3,1255.4,40.3,2.1,21.5,29.4


Let's see what is the most similar recipe, in terms of nutritions, to our most sematically similar recipe.

In [46]:
top_similar_recipes[['CleanRecipeTextData','url']].iloc[1].values

array(['lunchsnackSteak Meat Sweet Brunch < 30 Mins Stove Top Easylbs ground beef onion soup mix, 1 packet tablespoons brown sugar, firmly packed cup water cup ketchup hamburger bunslarg skillet brown ground beef mediumhigh heat drain stir onion soup mix brown sugar water ketchup bring boil high heat reduc heat low simmer uncovered stir occasionally 8 minut mixtur thickens serv hamburg buns',
       'https://www.food.com/recipe/Country-Sloppy-Joes-43986'],
      dtype=object)

By inspecting the link (https://www.food.com/recipe/Country-Sloppy-Joes-43986), we realize that is *ALSO* a burger!! That's fun! We found a burger that is most nutritionally similar to the burger that's most semantically similar to our user prompt!

## Curating the Data for Streamlit  <a class ='author' id='curate'>

Now let's curate our dataset for the Streamlit app.

--------------------------------
**Streamlit** is an open-source Python library that allows you to create interactive web applications for data science and machine learning projects with minimal effort. It is designed to simplify the process of converting data scripts into shareable web apps, making it easier for data scientists and developers to showcase their work and allow others to interact with their data-driven applications.

Key features of Streamlit include:

- Ease of Use: Streamlit provides a simple and intuitive API that allows users to build web applications using familiar Python scripting. You can create interactive widgets, plots, and charts with just a few lines of code.

- Rapid Prototyping: Streamlit allows for rapid prototyping and iterative development, making it easy to experiment with different data visualizations and functionalities.

- Data Visualization: You can easily create charts, plots, and graphs using popular data visualization libraries like Matplotlib, Plotly, and Altair.

- Widgets: Streamlit provides a wide range of widgets like sliders, buttons, text input, and more, which allow users to interact with the app and change parameters dynamically.

- Sharing and Deployment: Once the app is developed, it can be easily shared with others by sharing the script or deploying it to a web server.

- Integration with Data Science Libraries: Streamlit seamlessly integrates with popular Python data science libraries like Pandas, NumPy, and scikit-learn, allowing you to leverage your existing data analysis skills.

- Customization: While Streamlit is designed for simplicity, it also allows for customization and extension, enabling more advanced users to create complex applications.
--------------------------------
The main issue with Streamlit is that it doesn't function well when the data it's operatin on is too heavy. So we'll have to downsize our data. Also, I'd like the Streamlit app to show some user reviews for the recipes. 

In what follows, we'll import the reviews dataset, attach the top-2 reviews associated with recipes (that have reviews) and then take a smaller portion of the resulting data to feed our Streamlit app.

In [43]:
recipes.sample()

,RecipeId,Name,AuthorId,AuthorName,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeInstructions,url,ingred_quants,ingred_items,YearPublished,MonthPublished,DayPublished,HourPublished,TotalMinutes,CorrectAggregatedRating,BerTopics,CleanRecipeTextData,num_tokens,ada_embeddings
103489,523648.0,Light Thai Coconut Soup,1803541378,Late Night Gourmet,You can make one of the world's most exotic so...,[https://img.sndimg.com/food/image/upload/w_55...,Fruit,"[Fruit, Thai, Asian, < 60 Mins]","[1, 1, 2, 1⁄4, 1, 4, 1, 2, 2, 1, 1, 1, 3, 3, 1...","[medium shrimp, carrot, garlic cloves, onion, ...",1.0,133.1,3.4,0.5,95.5,1155.2,12.8,1.7,3.5,13.7,6.0,"[If making the stock, combine carrots, garlic,...",https://www.food.com/recipe/Light-Thai-Coconut...,"[1, 1, 2, 1⁄4, 1, 4, 1, 2, 2, 1, 1, 1, 3, 3, 1...","[lb medium shrimp, peeled and deveined, reserv...",2015,9,21,14,40,5.0,0_asparagus_salmon_fillet,fruitFruit Thai Asian < 60 Minslb medium shrim...,213,"[0.02085382491350174, -0.010258411057293415, 0..."


In [44]:
# Remove the columns that won't be used in teh app; this way we can save some space

recipes_streamlit = recipes.drop('AuthorId Keywords RecipeIngredientQuantities RecipeIngredientParts num_tokens YearPublished MonthPublished DayPublished HourPublished BerTopics'.split(),axis=1).reset_index(drop=True)

In [45]:
recipes_streamlit.shape

(108907, 25)

In [46]:
reviews = pd.read_parquet('../reviews.parquet')

In [47]:
reviews.shape

(1401982, 8)

In [48]:
reviews.sample(0)

,ReviewId,RecipeId,AuthorId,AuthorName,Rating,Review,DateSubmitted,DateModified


In [49]:
recipes_streamlit.sample(0)

,RecipeId,Name,AuthorName,Description,Images,RecipeCategory,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeInstructions,url,ingred_quants,ingred_items,TotalMinutes,CorrectAggregatedRating,CleanRecipeTextData,ada_embeddings


In [50]:
merged_data=pd.merge(recipes_streamlit,reviews,on='RecipeId')

In [51]:
# Just keep the top two reviews for each recipe (due to size constraints)

merged_data = merged_data.groupby('RecipeId').head(2).reset_index(drop=True)

In [52]:
merged_data.head(10)

,RecipeId,Name,AuthorName_x,Description,Images,RecipeCategory,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeInstructions,url,ingred_quants,ingred_items,TotalMinutes,CorrectAggregatedRating,CleanRecipeTextData,ada_embeddings,ReviewId,AuthorId,AuthorName_y,Rating,Review,DateSubmitted,DateModified
0,38.0,Low-Fat Berry Blue Frozen Dessert,Dancer,Make and share this Low-Fat Berry Blue Frozen ...,[https://img.sndimg.com/food/image/upload/w_55...,Desserts,4.0,170.9,2.5,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,"[Toss 2 cups berries with sugar., Let stand fo...",https://www.food.com/recipe/Low-Fat-Berry-Blue...,"[4, 1⁄4, 1, 1]","[cups blueberries, fresh or frozen, cup granul...",285,4.25,dessertDessert Low Protein Low Cholesterol Hea...,"[-0.0026673052925616503, -0.007128490600734949...",579149,596506,allisonsmommy,5,"Yummy, yummy, yummy! I am a big fan of fruit a...",2008-02-13 00:26:49+00:00,2008-02-13 00:26:49+00:00
1,38.0,Low-Fat Berry Blue Frozen Dessert,Dancer,Make and share this Low-Fat Berry Blue Frozen ...,[https://img.sndimg.com/food/image/upload/w_55...,Desserts,4.0,170.9,2.5,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,"[Toss 2 cups berries with sugar., Let stand fo...",https://www.food.com/recipe/Low-Fat-Berry-Blue...,"[4, 1⁄4, 1, 1]","[cups blueberries, fresh or frozen, cup granul...",285,4.25,dessertDessert Low Protein Low Cholesterol Hea...,"[-0.0026673052925616503, -0.007128490600734949...",789264,1060485,allyop135,4,very good!! the directions were alittle weird ...,2009-01-15 20:02:30+00:00,2009-01-15 20:02:30+00:00
2,39.0,Biryani,elly9812,Make and share this Biryani recipe from Food.com.,[https://img.sndimg.com/food/image/upload/w_55...,Chicken,1.0,1110.7,58.8,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,[Soak saffron in warm milk for 5 minutes and p...,https://www.food.com/recipe/Biryani-39,"[1, 4, 2, 2, 8, 1⁄4, 8, 1⁄2, 1, 1, 1⁄4, 1⁄4, 1...","[tablespoon saffron, teaspoons milk, warm, hot...",265,3.00,chickenChicken Thigh & Leg Chicken Poultry Mea...,"[0.014597306959331036, 0.008418912068009377, -...",361851,393254,cassiopeia94,3,I have an Indian friend who made this dish for...,2007-01-20 14:00:09+00:00,2007-01-20 14:00:09+00:00
3,40.0,Best Lemonade,Stephen Little,This is from one of my first Good House Keepi...,[https://img.sndimg.com/food/image/upload/w_55...,Beverages,10.0,311.1,0.2,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,"[Into a 1 quart Jar with tight fitting lid, pu...",https://www.food.com/recipe/Best-Lemonade-40,"[1 1⁄2, 1, , 1 1⁄2, , 3⁄4]","[cups sugar, tablespoon lemons, rind of or 1 t...",35,4.33,beveragLow Protein Low Cholesterol Healthy Sum...,"[0.026460928842425346, -0.008153736591339111, ...",34854,59064,Missy Wombat,5,My favourite lemonade recipe is essentially th...,2002-12-18 03:38:25+00:00,2002-12-18 03:38:25+00:00
4,40.0,Best Lemonade,Stephen Little,This is from one of my first Good House Keepi...,[https://img.sndimg.com/food/image/upload/w_55...,Beverages,10.0,311.1,0.2,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,"[Into a 1 quart Jar with tight fitting lid, pu...",https://www.food.com/recipe/Best-Lemonade-40,"[1 1⁄2, 1, , 1 1⁄2, , 3⁄4]","[cups sugar, tablespoon lemons, rind of or 1 t...",35,4.33,beveragLow Protein Low Cholesterol Healthy Sum...,"[0.026460928842425346, -0.008153736591339111, ...",39999,70154,Snez Stadnik,1,complicated recipe but cool.,2003-02-01 19:29:37+00:00,2003-02-01 19:29:37+00:00
5,41.0,Carina's Tofu-Vegetable Kebabs,Cyclopz,This dish is best prepared a day in advance to...,[https://img.sndimg.com/food/image/upload/w_55...,Vegetarian/Vegan,2.0,536.1,24.0,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,"[Drain the tofu, carefully squeezing out exces...",https://www.food.com/recipe/Carina's-Tofu-Vege...,"[12, 1, 2, 1, 10, 1, 3, 2, 2, 2, 1, 2, 1⁄2, 1⁄...","[ounces extra firm tofu, water-packed, medium ...",260,4.50,vegetarianveganBeans Vegetable Low Cholesterol...,"[0.012521459721028805, -0.03002464771

In [53]:
# Concatenate the two reviews for each recipe into one -- separate them with ';;' for later

merged_data = merged_data.groupby('RecipeId')['Review'].agg(lambda x: ';;'.join(x)).reset_index()
merged_data

,RecipeId,Review
0,38.0,"Yummy, yummy, yummy! I am a big fan of fruit a..."
1,39.0,I have an Indian friend who made this dish for...
2,40.0,My favourite lemonade recipe is essentially th...
3,41.0,I thought this was very GOOD! ANYONE who is af...
4,42.0,This is a very good soup. We served ours warm...
...,...,...
90881,540263.0,"Wow, was this good. I loved the sweetness at t..."
90882,540370.0,"Wow, what an awesome drink. I knew that I woul..."
90883,540416.0,What a delicious way to serve fresh green bean...
90884,540470.0,"Wow, this was fabulous. But then, with kahlua ..."


In [54]:
recipes_streamlit.sample(0)

,RecipeId,Name,AuthorName,Description,Images,RecipeCategory,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeInstructions,url,ingred_quants,ingred_items,TotalMinutes,CorrectAggregatedRating,CleanRecipeTextData,ada_embeddings


In [55]:
recipes_streamlit = pd.merge(recipes_streamlit,merged_data,on='RecipeId')

In [56]:
recipes_streamlit

,RecipeId,Name,AuthorName,Description,Images,RecipeCategory,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeInstructions,url,ingred_quants,ingred_items,TotalMinutes,CorrectAggregatedRating,CleanRecipeTextData,ada_embeddings,Review
0,38.0,Low-Fat Berry Blue Frozen Dessert,Dancer,Make and share this Low-Fat Berry Blue Frozen ...,[https://img.sndimg.com/food/image/upload/w_55...,Desserts,4.0,170.9,2.5,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,"[Toss 2 cups berries with sugar., Let stand fo...",https://www.food.com/recipe/Low-Fat-Berry-Blue...,"[4, 1⁄4, 1, 1]","[cups blueberries, fresh or frozen, cup granul...",285,4.25,dessertDessert Low Protein Low Cholesterol Hea...,"[-0.0026673052925616503, -0.007128490600734949...","Yummy, yummy, yummy! I am a big fan of fruit a..."
1,39.0,Biryani,elly9812,Make and share this Biryani recipe from Food.com.,[https://img.sndimg.com/food/image/upload/w_55...,Chicken,1.0,1110.7,58.8,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,[Soak saffron in warm milk for 5 minutes and p...,https://www.food.com/recipe/Biryani-39,"[1, 4, 2, 2, 8, 1⁄4, 8, 1⁄2, 1, 1, 1⁄4, 1⁄4, 1...","[tablespoon saffron, teaspoons milk, warm, hot...",265,3.00,chickenChicken Thigh & Leg Chicken Poultry Mea...,"[0.014597306959331036, 0.008418912068009377, -...",I have an Indian friend who made this dish for...
2,40.0,Best Lemonade,Stephen Little,This is from one of my first Good House Keepi...,[https://img.sndimg.com/food/image/upload/w_55...,Beverages,10.0,311.1,0.2,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,"[Into a 1 quart Jar with tight fitting lid, pu...",https://www.food.com/recipe/Best-Lemonade-40,"[1 1⁄2, 1, , 1 1⁄2, , 3⁄4]","[cups sugar, tablespoon lemons, rind of or 1 t...",35,4.33,beveragLow Protein Low Cholesterol Healthy Sum...,"[0.026460928842425346, -0.008153736591339111, ...",My favourite lemonade recipe is essentially th...
3,41.0,Carina's Tofu-Vegetable Kebabs,Cyclopz,This dish is best prepared a day in advance to...,[https://img.sndimg.com/food/image/upload/w_55...,Vegetarian/Vegan,2.0,536.1,24.0,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,"[Drain the tofu, carefully squeezing out exces...",https://www.food.com/recipe/Carina's-Tofu-Vege...,"[12, 1, 2, 1, 10, 1, 3, 2, 2, 2, 1, 2, 1⁄2, 1⁄...","[ounces extra firm tofu, water-packed, medium ...",260,4.50,vegetarianveganBeans Vegetable Low Cholesterol...,"[0.012521459721028805, -0.03002464771270752, -...",I thought this was very GOOD! ANYONE who is af...
4,42.0,Cabbage Soup,Duckie067,Make and share this Cabbage Soup recipe from F...,[https://img.sndimg.com/food/image/upload/w_55...,Vegetables,11.0,103.6,0.4,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,"[Mix everything together and bring to a boil.,...",https://www.food.com/recipe/Cabbage-Soup-42,"[46, 4, 1, 2, 1]","[ounces plain tomato juice, cups cabbage, shre...",50,2.67,vegetLow Protein Vegan Low Cholesterol Healthy...,"[0.015012815594673157, -0.01064545102417469, 0...",This is a very good soup. We served ours warm...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90881,540263.0,Ho Chi Minh Limeade,Chef PotPie,"A refreshing alternative to Vietnamese coffee,...",[https://img.sndimg.com/food/image/upload/w_55...,Beverages,3.0,256.6,6.7,4.2,26.0,110.5,45.3,0.2,42.4,6.2,1.0,"[In a highball glass, combine all the ingredie...",https://www.food.com/recipe/Ho-Chi-Minh-Limead...,"[1, 1⁄4, 1⁄4, ]","[lime, juice of, cup sweetened condensed milk,...",5,5.00,"beveragHealthy < 15 Mins Easylime, juice of cu...","[-0.0063337292522192, -0.004324314650148153, 0...","Wow, was this good. I loved the sweetness at t..."
90882,540370.0,Call Me a Cab (Lemonade Vodka Wine Cocktail),Chef PotPie,This is a lovely summertime cocktail that is n...,[https://img.sndimg.com/food/image/upload/w_55...,Beverages,3.0,68.0,0.1,0.0,0.0,6.8,17.7,0.0,17.0,0.1,1.0,"[Place ice in your desired cocktail glass., Po...",

Let's now drop the null values:

In [57]:
recipes_streamlit.isna().sum()

RecipeId                     0
Name                         0
AuthorName                   0
Description                  0
Images                       0
RecipeCategory               0
ReviewCount                  0
Calories                     0
FatContent                   0
SaturatedFatContent          0
CholesterolContent           0
SodiumContent                0
CarbohydrateContent          0
FiberContent                 0
SugarContent                 0
ProteinContent               0
RecipeServings               0
RecipeInstructions           0
url                          0
ingred_quants                0
ingred_items                 0
TotalMinutes                 0
CorrectAggregatedRating    642
CleanRecipeTextData          0
ada_embeddings               0
Review                       0
dtype: int64

In [58]:
recipes_streamlit.dropna(subset=['CorrectAggregatedRating'],inplace=True)

In [59]:
recipes_streamlit.isna().sum()

RecipeId                   0
Name                       0
AuthorName                 0
Description                0
Images                     0
RecipeCategory             0
ReviewCount                0
Calories                   0
FatContent                 0
SaturatedFatContent        0
CholesterolContent         0
SodiumContent              0
CarbohydrateContent        0
FiberContent               0
SugarContent               0
ProteinContent             0
RecipeServings             0
RecipeInstructions         0
url                        0
ingred_quants              0
ingred_items               0
TotalMinutes               0
CorrectAggregatedRating    0
CleanRecipeTextData        0
ada_embeddings             0
Review                     0
dtype: int64

Also, we don't want empty strings as reviews:

In [60]:
recipes_streamlit[recipes_streamlit['Review'] == '']

,RecipeId,Name,AuthorName,Description,Images,RecipeCategory,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeInstructions,url,ingred_quants,ingred_items,TotalMinutes,CorrectAggregatedRating,CleanRecipeTextData,ada_embeddings,Review
25202,137012.0,No Bake Blueberry Cheesecake,fawn512,"these are soo good, and easy to make since you...",[https://img.sndimg.com/food/image/upload/w_55...,Desserts,2.0,386.3,20.8,8.4,37.7,302.3,48.3,0.4,24.4,4.0,8.0,"[Combine crumbs and sugar, pour melted butter....",https://www.food.com/recipe/No-Bake-Blueberry-...,"[1 1⁄2, 2, 1⁄3, 1, 1, 1⁄4, 1, 3, 1]","[cups graham cracker crumbs, tablespoons brown...",30,5.0,dessertDessert Cheese < 30 Mins Easycups graha...,"[-0.007179928943514824, 0.0027766393031924963,...",
82458,454118.0,Chili Roasted Salmon With Cilantro Cream,Sammy Mae,Make and share this Chili Roasted Salmon With ...,[https://img.sndimg.com/food/image/upload/v1/i...,Regional,1.0,319.4,12.8,3.0,71.2,149.0,20.1,3.4,2.1,32.4,4.0,[Preheat the oven to 425 degrees. Mist 2 large...,https://www.food.com/recipe/Chili-Roasted-Salm...,"[, 4, 4, 2, 2, 3, 4, 1 1⁄2, 1, 1, 1⁄2, , , 1⁄3...","[nonstick cooking spray, tablespoons fresh lim...",20,5.0,regionSouthwestern U.S. Mexican Low Cholestero...,"[0.006318054627627134, 0.005944670177996159, -...",
90602,533896.0,Christmas Morning Breakfast Tarts,Food.com,Make and share this Christmas Morning Breakfas...,[https://img.sndimg.com/food/image/upload/w_55...,Desserts,1.0,456.7,32.3,9.6,196.1,301.1,28.2,0.9,0.7,13.0,4.0,[Roll out the puff pastry sheet; cut into 4 sq...,https://www.food.com/recipe/Christmas-Morning-...,"[1, 4, 2, 8, 1⁄4]","[sheet puff pastry (store-bought), eggs, slice...",58,5.0,dessertBreakfast Dessert Christmas Brunch < 60...,"[0.005176373291760683, -0.0030061332508921623,...",


In [61]:
recipes_streamlit = recipes_streamlit[recipes_streamlit['Review'] != '']

In [62]:
recipes_streamlit

,RecipeId,Name,AuthorName,Description,Images,RecipeCategory,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeInstructions,url,ingred_quants,ingred_items,TotalMinutes,CorrectAggregatedRating,CleanRecipeTextData,ada_embeddings,Review
0,38.0,Low-Fat Berry Blue Frozen Dessert,Dancer,Make and share this Low-Fat Berry Blue Frozen ...,[https://img.sndimg.com/food/image/upload/w_55...,Desserts,4.0,170.9,2.5,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,"[Toss 2 cups berries with sugar., Let stand fo...",https://www.food.com/recipe/Low-Fat-Berry-Blue...,"[4, 1⁄4, 1, 1]","[cups blueberries, fresh or frozen, cup granul...",285,4.25,dessertDessert Low Protein Low Cholesterol Hea...,"[-0.0026673052925616503, -0.007128490600734949...","Yummy, yummy, yummy! I am a big fan of fruit a..."
1,39.0,Biryani,elly9812,Make and share this Biryani recipe from Food.com.,[https://img.sndimg.com/food/image/upload/w_55...,Chicken,1.0,1110.7,58.8,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,[Soak saffron in warm milk for 5 minutes and p...,https://www.food.com/recipe/Biryani-39,"[1, 4, 2, 2, 8, 1⁄4, 8, 1⁄2, 1, 1, 1⁄4, 1⁄4, 1...","[tablespoon saffron, teaspoons milk, warm, hot...",265,3.00,chickenChicken Thigh & Leg Chicken Poultry Mea...,"[0.014597306959331036, 0.008418912068009377, -...",I have an Indian friend who made this dish for...
2,40.0,Best Lemonade,Stephen Little,This is from one of my first Good House Keepi...,[https://img.sndimg.com/food/image/upload/w_55...,Beverages,10.0,311.1,0.2,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,"[Into a 1 quart Jar with tight fitting lid, pu...",https://www.food.com/recipe/Best-Lemonade-40,"[1 1⁄2, 1, , 1 1⁄2, , 3⁄4]","[cups sugar, tablespoon lemons, rind of or 1 t...",35,4.33,beveragLow Protein Low Cholesterol Healthy Sum...,"[0.026460928842425346, -0.008153736591339111, ...",My favourite lemonade recipe is essentially th...
3,41.0,Carina's Tofu-Vegetable Kebabs,Cyclopz,This dish is best prepared a day in advance to...,[https://img.sndimg.com/food/image/upload/w_55...,Vegetarian/Vegan,2.0,536.1,24.0,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,"[Drain the tofu, carefully squeezing out exces...",https://www.food.com/recipe/Carina's-Tofu-Vege...,"[12, 1, 2, 1, 10, 1, 3, 2, 2, 2, 1, 2, 1⁄2, 1⁄...","[ounces extra firm tofu, water-packed, medium ...",260,4.50,vegetarianveganBeans Vegetable Low Cholesterol...,"[0.012521459721028805, -0.03002464771270752, -...",I thought this was very GOOD! ANYONE who is af...
4,42.0,Cabbage Soup,Duckie067,Make and share this Cabbage Soup recipe from F...,[https://img.sndimg.com/food/image/upload/w_55...,Vegetables,11.0,103.6,0.4,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,"[Mix everything together and bring to a boil.,...",https://www.food.com/recipe/Cabbage-Soup-42,"[46, 4, 1, 2, 1]","[ounces plain tomato juice, cups cabbage, shre...",50,2.67,vegetLow Protein Vegan Low Cholesterol Healthy...,"[0.015012815594673157, -0.01064545102417469, 0...",This is a very good soup. We served ours warm...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90881,540263.0,Ho Chi Minh Limeade,Chef PotPie,"A refreshing alternative to Vietnamese coffee,...",[https://img.sndimg.com/food/image/upload/w_55...,Beverages,3.0,256.6,6.7,4.2,26.0,110.5,45.3,0.2,42.4,6.2,1.0,"[In a highball glass, combine all the ingredie...",https://www.food.com/recipe/Ho-Chi-Minh-Limead...,"[1, 1⁄4, 1⁄4, ]","[lime, juice of, cup sweetened condensed milk,...",5,5.00,"beveragHealthy < 15 Mins Easylime, juice of cu...","[-0.0063337292522192, -0.004324314650148153, 0...","Wow, was this good. I loved the sweetness at t..."
90882,540370.0,Call Me a Cab (Lemonade Vodka Wine Cocktail),Chef PotPie,This is a lovely summertime cocktail that is n...,[https://img.sndimg.com/food/image/upload/w_55...,Beverages,3.0,68.0,0.1,0.0,0.0,6.8,17.7,0.0,17.0,0.1,1.0,"[Place ice in your desired cocktail glass., Po...",

In [63]:
# Turn the concatenated top-two reviews into a list of two reviews

recipes_streamlit['Review'] = recipes_streamlit['Review'].apply(lambda x: x.split(';;'))

C:\Users\mathe\AppData\Local\Temp\ipykernel_11300\444145016.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recipes_streamlit['Review'] = recipes_streamlit['Review'].apply(lambda x: x.split(';;'))


We'll just use 10% of our data to power our Streamlit app. This won't matter much since the goal is to showcase some cool possibilities, not to actually deploy a high-performance model for production.

**NOTE:** I have tried bigger portions but the app keeps crashing.

In [67]:
recipes_streamlit_10 = recipes_streamlit.sample(frac=.1)

In [68]:
recipes_streamlit_10.shape

(9024, 26)

In [69]:
recipes_streamlit_10.to_pickle('recipes_reviews_streamlit_10.pkl')

## Conclusion <a class ='author' id='conc'>

We created two recommender systems: 
- one that recipes a user prompt consisting of recipe ingredients or instructions and returns the top-5 recipes that are most similar to it.
- another one that takes a recipe from the recipes dataset, and finds the top recipes (also from the recipes dataset) that are most similar to it in terms of nutritional facts.

We showed that these two recommender systems can be combined into a single recommender system, and illustarted a use-case for that.

Finally, we curated our recipes data -- enhanced with user reviews assosciated with recipes -- to power-up a live recommender system using Streamlit. The links to the app will be shared in the GitHub repository of the project.